In [1]:
import tictactoe as ttt
import cv2
import numpy as np

In [2]:

def tictactoeState(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    resized_original_image = cv2.resize(image, (480, 480))
    img = ttt.pre_processing(resized_original_image)
    img = ttt.build_around_to_500(img)
    contours = ttt.find_contours(img, 3)
    contours = ttt.move_contours_back_to_480(contours)
    state = ttt.find_digital_game_array(contours, ttt.recognize_contour_type)
    return state

tictactoeState(cv2.imread('46_cropped_whiteboard.jpg'))



[[0, -1, 0], [-1, 1, 0], [0, -1, 1]]